# Customizing Model.fit()

## 1. Imports and Configuration

In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# Configure GPU memory growth to be dynamic instead of allocating all memory at once
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)

## 2. Data Loading and Preprocessing

In [2]:
from tensorflow.keras.datasets import mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.reshape(-1, 28, 28, 1).astype("float32") / 255.0
x_test = x_test.reshape(-1, 28, 28, 1).astype("float32") / 255.0

## 3. Model Definition

In [3]:
model = keras.Sequential(
    [
        layers.Input(shape=(28, 28, 1)),
        layers.Conv2D(64, (3, 3), padding="same"),
        layers.ReLU(),
        layers.Conv2D(128, (3, 3), padding="same"),
        layers.ReLU(),
        layers.Flatten(),
        layers.Dense(10),
    ],
    name="model",
)

### Custom Fit

In cases you might need a custom fit method can be when you are working with GANs. In this case, you might want to train the discriminator and generator separately. In this case, you can define a custom fit method that trains the discriminator and generator separately. etc..

In [4]:
class CustomFit(keras.Model):
    def __init__(self, model):
        # super to inherit from keras.Model
        super(CustomFit, self).__init__()
        self.model = model

    def compile(self, optimizer, loss):
        super(CustomFit, self).compile()
        self.optimizer = optimizer
        self.loss = loss

    def train_step(self, data):
        x, y = data

        # We do the forward pass and the loss computation, when we do it
        # inside the GradientTape context, we are tracing/recording those operations.
        # It is used later, when we do the backpropagation step 
        # (tape.gradient(loss, trainable_vars)).
        with tf.GradientTape() as tape:
            # Caclulate predictions
            y_pred = self.model(x, training=True)

            # Loss
            loss = self.loss(y, y_pred)

        # Gradients
        training_vars = self.trainable_variables
        gradients = tape.gradient(loss, training_vars)

        # Step with optimizer
        self.optimizer.apply_gradients(zip(gradients, training_vars))
        acc_metric.update_state(y, y_pred)

        return {"loss": loss, "accuracy": acc_metric.result()}
    
    def test_step(self, data):
        # Unpack the data
        x, y = data

        # Compute predictions
        y_pred = self.model(x, training=False)

        # Updates the metrics tracking the loss
        loss = self.loss(y, y_pred)

        # Update the metrics.
        acc_metric.update_state(y, y_pred)
        return {"loss": loss, "accuracy": acc_metric.result()}


acc_metric = keras.metrics.SparseCategoricalAccuracy(name="accuracy")

## 4. Compile Model

In [5]:
training = CustomFit(model)
training.compile(
    optimizer=keras.optimizers.Adam(learning_rate=3e-4),
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
)

## 5. Model Training and Evaluation

In [6]:
# .fit() is going to be done by the CustomFit class. Hence
# we do fit in a custom way. Fit calls train_step internally
training.fit(x_train, y_train, batch_size=64, epochs=2)

# evaluate calls test_step internally
training.evaluate(x_test, y_test, batch_size=64)

Epoch 1/2
938/938 [==============================] - 21s 14ms/step - loss: 0.1725 - accuracy: 0.8991
Epoch 2/2
157/157 [==============================] - 4s 22ms/step - loss: 0.0508 - accuracy: 0.9670


[0.9677307605743408, 0.001281227101571858]